In [93]:
import os
import json

from langchain_openai import AzureOpenAIEmbeddings
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents._generated.models import QueryCaptionResult

from openai import AzureOpenAI

from dotenv import load_dotenv

load_dotenv()

class AzureEmbeddings:

    def __init__(self):
        pass

    @staticmethod
    def get_embedding():
        return AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

    @staticmethod
    def generate_embeddings(content: str):
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

        doc_result = embeddings.embed_documents([content])

        return doc_result[0]

openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"), 
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    )

embeddings_client = AzureEmbeddings()
store_search_url: str = f'https://{os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME')}.search.windows.net'
search_client = SearchClient(
            store_search_url, os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME"),
            AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"))
        )

In [84]:
user_input = "¿Cuáles son las causales de agravación del homicidio?"

In [85]:
query_system_template = f"Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base.\\n    You have access to Azure AI Search index with 100's of documents.\\n    Generate a search query based on the conversation and the new question.\\n    Do not include cited source filenames and document names e.g info.txt or doc.pdf in the search query terms.\\n    Do not include any text inside [] or <<>> in the search query terms.\\n    Do not include any special characters like '+'.\\n    If the question is not in English, translate the question to English before generating the search query.\\n    If you cannot generate a search query, return just the number 0."

query_few_shots = [
    {"role": "user", "content": "¿Qué dice el artículo 103 del Código Penal?"},
    {"role": "assistant", "content": '"Artículo 103" Ley 599 de 2000'},
    {
        "role": "user",
        "content": "¿Cuáles son las causales de agravación del homicidio?",
    },
    {
        "role": "assistant",
        "content": '"Causales de agravación del homicidio" "Ley 599 de 2000"',
    },
    {"role": "user", "content": "Si A le roba el celular a B, ¿qué delito comete?"},
    {"role": "assistant", "content": '"Hurto" "Ley 599 de 2000" "Celular"'},
    {
        "role": "user",
        "content": "¿En qué artículo de la Constitución Política se consagra el principio de dignidad humana?",
    },
    {
        "role": "assistant",
        "content": '"Principio de dignidad humana" "Constitución Política de Colombia"',
    },
    {"role": "user", "content": "¿Cuáles son las formas de autoría en derecho penal?"},
    {
        "role": "assistant",
        "content": 'Formas de autoría en derecho penal "Ley 599 de 2000"',
    },
    {"role": "user", "content": "¿Cuándo se consuma el hurto?"},
    {"role": "assistant", "content": '"Hurtoconsumación" "Ley 599 de 2000"'},
    {
        "role": "assistant",
        "content": '"Principio de dignidad humana" "Constitución Política de Colombia"',
    },
    {
        "role": "user",
        "content": "¿Las personas jurídicas pueden ser sujetos pasivos de los delitos de injuria y calumnia?",
    },
    {
        "role": "assistant",
        "content": '"Personas jurídicas" "sujetos pasivos" "delitos de injuria y calumnia"',
    },
    {
        "role": "user",
        "content": "¿Se puede archivar una investigación cuando se encuentra que el procesado actuó en legítima defensa?",
    },
    {
        "role": "assistant",
        "content": '"Archivo de investigación" "legítima defensa" "Ley 906 de 2004"',
    },
    {"role": "user", "content": "¿Cuáles son los requisitos de la denuncia?"},
    {"role": "assistant", "content": '"Requisitos de la denuncia" Ley 906 de 2004'},
]

query_user_content_template = f"Generate search query for: {user_input}"

query_messages = [
    {"role": "system", "content": query_system_template},
    *query_few_shots,
    {"role": "user", "content": query_user_content_template},
]

In [86]:
query = (
    openai_client.chat.completions.create(
        messages=query_messages,
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        temperature=0.0,
        n=1,
    )
    .choices[0]
    .message.content
)
# query = "Artículo 103 del Código Penal"

query_vectors = embeddings_client.generate_embeddings(content=query)

print("")
print("Query for:", query)


Query for: "Causales de agravación del homicidio" "Código Penal"


In [92]:
results = search_client.search(
    search_text=query,
    # filter=filters,
    top=5,
    query_type=QueryType.SEMANTIC,
    vector_queries=[
        {
            "vector": query_vectors,
            "k": 5,
            "fields": "content_vector",
            "kind": "vector",
            "exhaustive": True,
        }
    ],
    semantic_configuration_name="default",
    query_caption="extractive|highlight-false",
    scoring_profile="legal",
    scoring_parameters=[
        "tagx6-Constitucional",
        "tagx5-Legal",
        "tagx4-Infralegal",
        "tagx3-Jurisprudencia",
        "tagx2-Doctrina",
    ],
)

documents = []
printer_documents = []
for page in results.by_page():
    for document in page:
        captions: QueryCaptionResult = document["@search.captions"]
        doc = {
            # "score": document["@search.score"],
            # "id": document["id"],
            "content": document["content"],
            # "captions": captions[0].text,
            # "external_id": document["external_id"],
            "title": document["title"],
            "author": document["author"],
            # "page": document["page"],
            # "keywords": document["keywords"],
            "category": document["category"],
            "year": document["year"],
            # "has_copyright": document["has_copyright"],
        }
        documents.append(doc)
        printer_documents.append(json.dumps(doc, indent=4))

print("Legal profile - Query for: ", query)
print("=============================")
print("\n".join(printer_documents))

Legal profile - Query for:  "Causales de agravación del homicidio" "Código Penal"
{
    "content": "tiene un contenido culposo, los agravantes no\nUna estricta interpretaci\u00f3n gramatical de la\npueden ser las del tipo doloso.\nnorma, advierte que el art\u00edculo 105 del C\u00f3digo\nAl respecto precisa la Sala que en el tipo Penal, establece:\npreterintencional el sujeto activo dirige su\n[\u2026] Lo que obligatoriamente remite a los art\u00edculos\nvoluntad hacia la obtenci\u00f3n de un resultado\n103 y 104 del estatuto sustantivo (antes de las\nquerido, pero se produce uno m\u00e1s grave que se\nadiciones del 104.A.B). Advi\u00e9rtase, adem\u00e1s, que\nreprocha a t\u00edtulo de culpa porque ten\u00eda la\nlas causales de agravaci\u00f3n del homicidio culposo\ncapacidad de preverlo. Es una mixtura entre el\nse ubican en el art\u00edculo 110, siendo posteriores.\ndolo y la culpa y as\u00ed lo establece el art\u00edculo 24 del\nConcluyendo que el legislador no dej\u00f3 margen de\

In [88]:
response_system_template = f'Eres Ariel, un asistente para la investigación legal \n\n    Sé lo más detallado y preciso posible en tus respuestas. Cita el máximo número de fuentes posible, sin salirte del tema. En caso de encontrar información contradictoria, señálala y sugiere una posible causa. Expresa toda la información que encuentres en las fuentes proporcionadas. Solamente si la respuesta no está en las fuentes proporcionadas, responde "No encuentro información con esos términos, ¿puedes reformular tu consulta?". Al incluir títulos en tu respuesta, usa formato html (ej: titulos, <strong>).\n\nCada fuente tiene un nombre seguido por dos puntos y la información real, siempre incluye el nombre de la fuente para cada hecho que uses en la respuesta. Todas las fuentes son PDFs. Utiliza corchetes para referenciar la fuente, por ejemplo [info1.pdf]. No combines fuentes, lista cada fuente por separado, por ejemplo [info1.txt] [info2.pdf]. El formato de algunas fuentes puede incluir acentos, puntos o guiones. Asegúrate de capturar todo antes de ".pdf", por ejemplo: [SFC - Títulos valores electrónicos. Pagarés. Depósito centralizado de valores. Exigencia Concepto 2020086426-003 del 24 de junio de 2020 id2020086426.pdf]. No dividas los nombres de archivo por ninguna razón, ya que todos deben terminar en .pdf. Escribe la respuesta en formato HTML, pero sin incluir los tags "```html" al principio o final principio de tu respuesta.'

response_few_shots = []

response_user_content_template = f"{user_input} Sources: {documents}"

response_messages = [
    {"role": "system", "content": response_system_template},
    *query_few_shots,
    {"role": "user", "content": response_user_content_template},
]

In [89]:
response = (
    openai_client.chat.completions.create(
        messages=response_messages,
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        temperature=0.2,
        n=1,
    )
    .choices[0]
    .message.content
)

In [90]:
print(response)

<strong>Causales de agravación del homicidio</strong>

Las causales de agravación del homicidio están establecidas en el artículo 104 del Código Penal Colombiano (Ley 599 de 2000). A continuación, se detallan algunas de las causales de agravación:

1. **Cuando se comete por medio de otro delito**: Por ejemplo, el homicidio cometido en medio de un delito de peligro común que pueda ocasionar grave perjuicio para la comunidad o que genere afectación a la salud pública [CSJ - SCP Boletin Jurisprudencial 2021-07-16.pdf].

2. **Situación de indefensión o inferioridad de la víctima**: Cuando el agente coloca a la víctima en situación de indefensión o inferioridad o aprovecha esta situación [CSJ - SCP Boletin Jurisprudencial 2021-07-16.pdf].

3. **Parentesco**: La relación de parentesco entre el agresor y la víctima puede ser una circunstancia agravante [CSJ - SCP Boletin Jurisprudencial 2022-05-31.pdf].

4. **Indefensión de la víctima**: La indefensión de la víctima es una circunstancia que a